# Empire 2018 Guest Witness Allocation Algorithm
Developed by Viran Weerasekera (March 2018).

In [4]:

import gurobipy as grb
import numpy as np
import pandas as pd

In [5]:
def initImport(rd):
    
    global has_repeated
    has_repeated = {}

    has_repeated["same_do-opp"] = {}
    has_repeated["opp_do-same"] = {}
    has_repeated["opp_dt-opp"] = {}
    has_repeated["opp_do-opp"] = {}


    global conflicts
    conflicts = {}

    global teams
    teams = []
    global sides
    sides = {}
    global dts #dts = Destination Teams
    dts = {}
    global dos #dos = Destination Oppositions
    dos = {}
    data_csv = pd.read_csv("data_afterr{}.csv".format(rd),index_col=0)
    conflicts_csv = pd.read_csv("conflicts.csv",index_col=0)

    # Create empty conflict dictionaries
    for ind,row in data_csv.iterrows():
        teams.append(ind)
        sides[ind] = {}
        sides[ind][1] = row[1]
        sides[ind][2] = row[4]
        sides[ind][3] = row[7]
    
        dts[ind] = {}
        dts[ind][1] = row[2]
        dts[ind][2] = row[5]
        dts[ind][3] = row[8]
    
        dos[ind] = {}
        dos[ind][1] = row[3]
        dos[ind][2] = row[6]
        dos[ind][3] = row[9]
    
    
    
        conflicts["{}_p".format(ind)] = []
        conflicts["{}_d".format(ind)] = []

    conflicts_csv.head()
    for ind,row in conflicts_csv.iterrows():
        team1 = ind
        team2 = row[0]
        
        if (team1 != "nan") and (team2 != "nan"):
            #1p, 2p
            if "{}_p".format(team2) not in conflicts["{}_p".format(team1)]:
                conflicts["{}_p".format(team1)].append("{}_p".format(team2))
            #1d, 2p
            if "{}_p".format(team2) not in conflicts["{}_d".format(team1)]:
                conflicts["{}_d".format(team1)].append("{}_p".format(team2))
            #1p, 2d
            if "{}_d".format(team2) not in conflicts["{}_p".format(team1)]:
                conflicts["{}_p".format(team1)].append("{}_d".format(team2))
            #1d, 2d
            if "{}_d".format(team2) not in conflicts["{}_d".format(team1)]:
                conflicts["{}_d".format(team1)].append("{}_d".format(team2))
            
            #2p, 1p
            if "{}_p".format(team1) not in conflicts["{}_p".format(team2)]:
                conflicts["{}_p".format(team2)].append("{}_p".format(team1))
            #2d, 1p
            if "{}_p".format(team1) not in conflicts["{}_d".format(team2)]:
                conflicts["{}_d".format(team2)].append("{}_p".format(team1))
            #2p, 1d
            if "{}_d".format(team1) not in conflicts["{}_p".format(team2)]:
                conflicts["{}_p".format(team2)].append("{}_d".format(team1))
            #2d, 1d
            if "{}_d".format(team1) not in conflicts["{}_d".format(team2)]:
                conflicts["{}_d".format(team2)].append("{}_d".format(team1))
        #print(conflicts)
def addConflicts(rd):
    for team in teams:
        
        if rd >= 2:
            # Destination Team
            if dts[team][1] not in conflicts["{}_{}".format(team,sides[team][1])]:
                conflicts["{}_{}".format(team,sides[team][1])].append(dts[team][1])
                    
            # Destination's Opponent
            if dos[team][1] not in conflicts["{}_{}".format(team,sides[team][1])]:
                conflicts["{}_{}".format(team,sides[team][1])].append(dos[team][1])
            
        if rd >= 3:
            # Destination Team
            if dts[team][2] not in conflicts["{}_{}".format(team,sides[team][2])]:
                conflicts["{}_{}".format(team,sides[team][2])].append(dts[team][2])
                    
            # Destination's Opponent
            if dos[team][2] not in conflicts["{}_{}".format(team,sides[team][2])]:
                conflicts["{}_{}".format(team,sides[team][2])].append(dos[team][2])
     
        if rd >= 4:
            # Destination Team
            if dts[team][3] not in conflicts["{}_{}".format(team,sides[team][3])]:
                conflicts["{}_{}".format(team,sides[team][3])].append(dts[team][3])
                    
            # Destination's Opponent
            if dos[team][3] not in conflicts["{}_{}".format(team,sides[team][3])]:
                conflicts["{}_{}".format(team,sides[team][3])].append(dos[team][3])
            
def generateCosts(rd):
    global masterpairings
    global pairings
    global roomranks
    global costs
    global softconflictcosts
    global powercosts
    global cannotgoto
    global sendtorooms
    global receivefromrooms
    
    cannotgoto = {}
    sendtorooms = {}
    receivefromrooms = {}
    
    masterpairings = {}
    pairings = {}
    pairings[1] = {}
    pairings[2] = {}
    roomranks = {}
    costs = {}
    softconflictcosts = {}
    powercosts = {}
    venuesbyteam = {}
    
    div1pairings_csv = pd.read_csv("pairings_r{}_div1.csv".format(rd),index_col=0)
    div2pairings_csv = pd.read_csv("pairings_r{}_div2.csv".format(rd),index_col=0)
    
    for ind,row in div1pairings_csv.iterrows():
        pairings[1][row[0]] = [row[1],row[2]]
        masterpairings[row[0]] = [row[1],row[2]]
        roomranks[row[0]] = ind
        venuesbyteam[row[1]] = row[0]
        venuesbyteam[row[2]] = row[0]
        cannotgoto[row[1]] = []
        cannotgoto[row[2]] = []
        sendtorooms[row[0]] = []
        receivefromrooms[row[0]] = []
    
    for ind,row in div2pairings_csv.iterrows():
        pairings[2][row[0]] = [row[1],row[2]]
        masterpairings[row[0]] = [row[1],row[2]]
        roomranks[row[0]] = ind
        venuesbyteam[row[1]] = row[0]
        venuesbyteam[row[2]] = row[0]
        cannotgoto[row[1]] = []
        cannotgoto[row[2]] = []
        sendtorooms[row[0]] = []
        receivefromrooms[row[0]] = []

    #print(masterpairings)
    for div1room,div1pairing in pairings[1].iteritems():
        for div2room,div2pairing in pairings[2].iteritems():
            costs[div1room,div2room] = 0
            costs[div2room,div1room] = 0
            
            softconflictcosts[div1room,div2room] = 0
            softconflictcosts[div2room,div1room] = 0
            
            powercosts[div1room,div2room] = 0
            powercosts[div2room,div1room] = 0

    for assignment,cost in costs.iteritems():
        originPairing = masterpairings[assignment[0]]
        destinationPairing = masterpairings[assignment[1]]
        
        costs[assignment] += 100*np.absolute(roomranks[assignment[0]] - roomranks[assignment[1]])
        powercosts[assignment] += 100*np.absolute(roomranks[assignment[0]] - roomranks[assignment[1]])
    
        # Conflicts
        if destinationPairing[0] in conflicts["{}_p".format(originPairing[0])]:
            costs[assignment] = grb.GRB.INFINITY
            cannotgoto[originPairing[0]].append(venuesbyteam[destinationPairing[0]])
        if destinationPairing[0] in conflicts["{}_d".format(originPairing[1])]:
            costs[assignment] = grb.GRB.INFINITY
            cannotgoto[originPairing[1]].append(venuesbyteam[destinationPairing[0]])
        
        # Can't send if the destination D team is in the P/D team's conflicts
        if destinationPairing[1] in conflicts["{}_p".format(originPairing[0])]:
            costs[assignment] = grb.GRB.INFINITY
            cannotgoto[originPairing[0]].append(venuesbyteam[destinationPairing[1]])
        if destinationPairing[1] in conflicts["{}_d".format(originPairing[1])]:
            costs[assignment] = grb.GRB.INFINITY
            cannotgoto[originPairing[1]].append(venuesbyteam[destinationPairing[1]])
            
              
def allocate(rd):
    m = grb.Model("Guest Witness Allocation - Round {}".format(rd))
    x = {}
    for key,value in costs.iteritems():
        x[key] = m.addVar(vtype = grb.GRB.BINARY, obj = value, name = "x_{}".format(value))

    m.modelSense = grb.GRB.MINIMIZE
    
    div1Rooms = []
    div2Rooms = []
    
    div1Rooms = pairings[1].keys()
    div2Rooms = pairings[2].keys()

    for room in masterpairings.keys():
        if room in div1Rooms:
            m.addConstr(sum(x[room,room2] for room2 in div2Rooms) == 1)
            m.addConstr(sum(x[room2,room] for room2 in div2Rooms) == 1)
        elif room in div2Rooms:
            m.addConstr(sum(x[room,room2] for room2 in div1Rooms) == 1)
            m.addConstr(sum(x[room2,room] for room2 in div1Rooms) == 1)
    
    m.optimize()
    
    print("-----------------------------------".format(rd))
    print("Guest Witness Allocations - Round {}".format(rd))
    print("-----------------------------------".format(rd))
    
    roomsToPrint = []
    roomsToPrint = masterpairings.keys()
    roomsToPrint.sort()
    
    allocatedOrigins = []
    allocatedDestinations = []
    
    d1outputRooms = [["From","To"]]
    d2outputRooms = [["From","To"]]
    
    for room in roomsToPrint:
        if room in div1Rooms:
            for room2 in div2Rooms:
                if x[room,room2].x != 0:
                    print("{} sends GWs to {} ({})".format(room,room2,costs[room,room2]))
                    d1outputRooms.append([room,room2])
                    if room in allocatedOrigins:
                        print("origin")
                    else:
                        allocatedOrigins.append(room)
                        
                    if room2 in allocatedDestinations:
                        print("destination")
                    else:
                        allocatedDestinations.append(room2)     
        elif room in div2Rooms:
            for room2 in div1Rooms:
                if x[room,room2].x != 0:
                    print("{} sends GWs to {} ({})".format(room,room2,costs[room,room2]))
                    d2outputRooms.append([room,room2])
                    if room in allocatedOrigins:
                        print("origin")
                    else:
                        allocatedOrigins.append(room)
                        
                    if room2 in allocatedDestinations:
                        print("destination")
                    else:
                        allocatedDestinations.append(room2)
    pdOutputArray1 = pd.DataFrame(d1outputRooms)
    pdOutputArray2 = pd.DataFrame(d2outputRooms)
    pdOutputArray1.to_csv("assignments_r{}_div1.csv".format(rd))
    pdOutputArray2.to_csv("assignments_r{}_div2.csv".format(rd))
      
def diagnosticReport():
    print("------------")
    print("Teams can send to the following rooms")
    for assignment,cost in costs.iteritems():
        if cost <= 100000:
            sendtorooms[assignment[0]].append(assignment[1])
            receivefromrooms[assignment[1]].append(assignment[0])
    for key,value in receivefromrooms.iteritems():
        if len(value) == 0:
            print("{} has nowhere to receive from".format(key))
    for key,value in sendtorooms.iteritems():
        if len(value) == 0:
            print("{} has nowhere to send to".format(key))
    print("------------")
    for key,value in cannotgoto.iteritems():
        print(key,value)
    print("------")
    print("Possible Destinations:")
    for key,value in sendtorooms.iteritems():
        print(key,value)
    print("------")
    print("Possible Recipients:")
    for key,value in receivefromrooms.iteritems():
        print(key,value)

def manualConflict(a,b):
    costs[(a,b)] = grb.GRB.INFINITY

In [6]:
nextRound = 4
lastRound = nextRound - 1

In [7]:
initImport(lastRound)
addConflicts(nextRound)
generateCosts(nextRound)
# Add manual conflicts (reciprocate)
manualConflict("15A","15D")
manualConflict("15D","15A")
manualConflict("15D","15A")
manualConflict("9B","15D")
manualConflict("15D","9B")
manualConflict("12A","21D")
manualConflict("21D","12A")

#print(costs)
allocate(nextRound)

Optimize a model with 40 rows, 200 columns and 400 nonzeros
Variable types: 0 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+100]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 20 rows and 100 columns
Presolve time: 0.02s
Presolved: 20 rows, 100 columns, 200 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)
Found heuristic solution: objective 2200.0000000

Root relaxation: objective 2.000000e+03, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2000.0000000 2000.00000  0.00%     -    0s

Explored 0 nodes (14 simplex iterations) in 0.07 seconds
Thread count was 4 (of 4 available processors)

S

In [8]:
diagnosticReport()

------------
Teams can send to the following rooms
------------
('Lemon', ['20A', '21D'])
('Tribbiani', ['14A', '9B'])
('Ricardo', ['15D', '21C'])
('Jefferson', ['11B', '14A'])
('Diaz', ['18A', '15D'])
('Stinson', ['11B', '14A'])
('Santiago', ['18D', '18A'])
('Benes', ['14B', '15C'])
('Sheffield', ['18D', '18B'])
('Scherbatsky', ['15A', '9B'])
('Buffay', ['14D', '9B'])
('Newman', ['14D', '14B'])
('Jordan', ['12A', '11D'])
('McFarland', ['14D', '14B'])
('Jeffords', ['20A', '18A'])
('Buchman', ['21C', '18A'])
('Parcell', ['11A', '14A'])
('Mertz', ['9B', '14A'])
('Bing', ['18D', '18B'])
('Peralta', ['20C', '20D'])
('Costanza', ['11B', '11D'])
('Green', ['20D', '18B'])
('Mosby', ['18D', '18A'])
('Walker', ['21D', '15D'])
('Janice', ['18A', '20A'])
('Channing', ['21D', '20A'])
('Eriksen', ['14D', '14B'])
('Hornberger', ['20A', '18C'])
('Kramer', ['20D', '21C'])
('Gunther', ['11D', '15A'])
('Adler', ['18C', '18D'])
('Seinfeld', ['14D', '12A'])
('Geller', ['15A', '11D'])
('Donaghy', ['12A', '

In [ ]:
print(pairings)